## NRT

This notebook runs all the preprocessing steps using NRT data for a specific region and time. 

In [1]:
import datetime
import pandas as pd
import geopandas as gpd

import FireIO, FireMain, FireTime, FireObj, FireConsts, postprocess, preprocess
from utils import timed

tst = [2023, 8, 28, 'AM']
ted = [2023, 9, 6, 'AM']
region = ('WesternUS',[-125.698046875,31.676476158707615,
                       -101.00078125,49.51429477264348])
list_of_ts = list(FireTime.t_generator(tst, ted))

## Preprocess the region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [2]:
preprocess.preprocess_region(region)

2024-03-26 17:01:46,293 - FireLog - INFO - Preprocessing has already occurred for this region.
2024-03-26 17:01:46,293 - FireLog - INFO - func:preprocess_region took: 1.52 ms


'data/FEDSpreprocessed/WesternUS/WesternUS.json'

## Are preprocessed input files available?

Check to see if all the files that we need have already been processed and are available on s3.

In [5]:
ts_that_need_preprocessing = set([
    *preprocess.check_preprocessed_file(tst, ted, sat="NOAA20", freq="NRT", location="s3"),
    *preprocess.check_preprocessed_file(tst, ted, sat="SNPP", freq="NRT", location="s3")
])

In [6]:
assert len(ts_that_need_preprocessing) == 0, "All input fires should be on s3"

## Preprocess for region and t

Do initial filtering and clustering using the preprocessed region (from local) and the half day files (from s3).

In [8]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, sensor="VIIRS", region=region, read_location="s3", read_region_location="local")

2024-03-26 17:03:09,044 - FireLog - INFO - func:read_region took: 193.62 ms
2024-03-26 17:03:09,045 - FireLog - INFO - filtering and clustering 2023-8-28 AM, VIIRS, WesternUS
2024-03-26 17:03:09,615 - FireLog - INFO - func:read_preprocessed_input took: 569.50 ms
2024-03-26 17:03:09,837 - FireLog - INFO - func:read_preprocessed_input took: 220.85 ms
2024-03-26 17:04:08,320 - FireLog - INFO - func:do_clustering took: 509.26 ms
2024-03-26 17:04:08,427 - FireLog - INFO - func:preprocess_region_t took: 59.58 sec
2024-03-26 17:04:08,614 - FireLog - INFO - func:read_region took: 185.85 ms
2024-03-26 17:04:08,614 - FireLog - INFO - filtering and clustering 2023-8-28 PM, VIIRS, WesternUS
2024-03-26 17:04:09,267 - FireLog - INFO - func:read_preprocessed_input took: 651.60 ms
2024-03-26 17:04:09,535 - FireLog - INFO - func:read_preprocessed_input took: 268.06 ms
2024-03-26 17:04:32,132 - FireLog - INFO - func:do_clustering took: 18.73 ms
2024-03-26 17:04:32,208 - FireLog - INFO - func:preprocess_

CPU times: user 4min 22s, sys: 700 ms, total: 4min 22s
Wall time: 4min 54s


## Run the fire expansion and merging algorithm

Now that we have all the region-specific preprocessed fires, we can go ahead and run `Fire_Forward`.

In [2]:
allfires, allpixels = FireMain.Fire_Forward(tst=tst, ted=ted, region=region, read_location="local")

2024-03-26 17:08:50,650 - FireLog - INFO - func:read_preprocessed took: 17.51 ms
2024-03-26 17:08:50,663 - FireLog - INFO - func:read_preprocessed took: 12.57 ms
2024-03-26 17:08:50,678 - FireLog - INFO - func:read_preprocessed took: 14.06 ms
2024-03-26 17:08:50,688 - FireLog - INFO - func:read_preprocessed took: 9.17 ms
2024-03-26 17:08:50,700 - FireLog - INFO - func:read_preprocessed took: 11.88 ms
2024-03-26 17:08:50,713 - FireLog - INFO - func:read_preprocessed took: 12.79 ms
2024-03-26 17:08:50,728 - FireLog - INFO - func:read_preprocessed took: 14.26 ms
2024-03-26 17:08:50,733 - FireLog - INFO - func:read_preprocessed took: 4.85 ms
2024-03-26 17:08:50,738 - FireLog - INFO - func:read_preprocessed took: 4.92 ms
2024-03-26 17:08:50,743 - FireLog - INFO - func:read_preprocessed took: 4.28 ms
2024-03-26 17:08:50,748 - FireLog - INFO - func:read_preprocessed took: 4.79 ms
2024-03-26 17:08:50,753 - FireLog - INFO - func:read_preprocessed took: 4.27 ms
2024-03-26 17:08:50,758 - FireLog 

We can save these files off in case something goes wrong later

In [3]:
postprocess.save_allpixels(allpixels, tst, ted, region)
postprocess.save_allfires_gdf(allfires.gdf, tst, ted, region)

2024-03-26 17:14:58,175 - FireLog - INFO - func:save_allpixels took: 234.29 ms
2024-03-26 17:14:58,364 - FireLog - INFO - func:save_allfires_gdf took: 187.83 ms


'data/FEDSoutput-s3-conus/WesternUS/2023/allfires_20230906_AM.parq'

## Write Snapshot files

Start by reading in the outputs from the run:

In [2]:
allpixels = postprocess.read_allpixels(tst, ted, region, location="local")
allfires_gdf = postprocess.read_allfires_gdf(tst, ted, region, location="local")

2024-03-26 17:21:38,182 - FireLog - INFO - func:read_allpixels took: 51.24 ms
2024-03-26 17:21:38,327 - FireLog - INFO - func:read_allfires_gdf took: 143.50 ms


Then iterate through all of the ts and save the snapshot layers.

In [3]:
%%time
postprocess.save_snapshots(allfires_gdf, region, tst, ted)

2024-03-26 17:20:01,846 - FireLog - INFO - func:save_snapshot_layers took: 433.92 ms
2024-03-26 17:20:02,304 - FireLog - INFO - func:save_snapshot_layers took: 454.04 ms
2024-03-26 17:20:02,798 - FireLog - INFO - func:save_snapshot_layers took: 490.36 ms
2024-03-26 17:20:03,296 - FireLog - INFO - func:save_snapshot_layers took: 494.19 ms
2024-03-26 17:20:03,838 - FireLog - INFO - func:save_snapshot_layers took: 538.16 ms
2024-03-26 17:20:04,393 - FireLog - INFO - func:save_snapshot_layers took: 550.41 ms
2024-03-26 17:20:04,972 - FireLog - INFO - func:save_snapshot_layers took: 575.37 ms
2024-03-26 17:20:05,554 - FireLog - INFO - func:save_snapshot_layers took: 579.09 ms
2024-03-26 17:20:06,142 - FireLog - INFO - func:save_snapshot_layers took: 583.98 ms
2024-03-26 17:20:06,762 - FireLog - INFO - func:save_snapshot_layers took: 616.14 ms
2024-03-26 17:20:07,405 - FireLog - INFO - func:save_snapshot_layers took: 638.92 ms
2024-03-26 17:20:08,042 - FireLog - INFO - func:save_snapshot_lay

CPU times: user 7.1 s, sys: 191 ms, total: 7.29 s
Wall time: 11.5 s


## Write Largefire files

First we will find all the large fires:

In [3]:
large_fires = postprocess.find_largefires(allfires_gdf)

2024-03-26 17:21:41,878 - FireLog - INFO - func:find_largefires took: 5.79 ms


First we'll use the `allpixels` object to create the `nplist` layer

In [4]:
%%time
postprocess.save_large_fires_nplist(allpixels, region, large_fires, tst)

2024-03-26 17:21:42,814 - FireLog - INFO - func:save_fire_nplist took: 77.39 ms
2024-03-26 17:21:42,855 - FireLog - INFO - func:save_fire_nplist took: 39.39 ms
2024-03-26 17:21:42,908 - FireLog - INFO - func:save_fire_nplist took: 50.98 ms
2024-03-26 17:21:42,963 - FireLog - INFO - func:save_fire_nplist took: 53.26 ms
2024-03-26 17:21:43,014 - FireLog - INFO - func:save_fire_nplist took: 48.82 ms
2024-03-26 17:21:43,062 - FireLog - INFO - func:save_fire_nplist took: 46.87 ms
2024-03-26 17:21:43,107 - FireLog - INFO - func:save_fire_nplist took: 42.90 ms
2024-03-26 17:21:43,154 - FireLog - INFO - func:save_fire_nplist took: 45.88 ms
2024-03-26 17:21:43,208 - FireLog - INFO - func:save_fire_nplist took: 52.34 ms
2024-03-26 17:21:43,254 - FireLog - INFO - func:save_fire_nplist took: 43.89 ms
2024-03-26 17:21:43,301 - FireLog - INFO - func:save_fire_nplist took: 45.06 ms
2024-03-26 17:21:43,355 - FireLog - INFO - func:save_fire_nplist took: 52.92 ms
2024-03-26 17:21:43,413 - FireLog - INFO

CPU times: user 1.05 s, sys: 64.8 ms, total: 1.12 s
Wall time: 2.24 s


The rest of the layers will be created directly from the `allfires_gdf` and have some merge handling

In [5]:
%%time
postprocess.save_large_fires_layers(allfires_gdf, region, large_fires, tst)

2024-03-26 17:21:45,118 - FireLog - INFO - func:save_fire_layers took: 132.35 ms


45 rows that potentially need a merge


2024-03-26 17:21:45,240 - FireLog - INFO - func:save_fire_layers took: 120.91 ms
2024-03-26 17:21:45,348 - FireLog - INFO - func:save_fire_layers took: 106.30 ms
2024-03-26 17:21:45,375 - FireLog - INFO - func:merge_rows took: 24.94 ms
2024-03-26 17:21:45,481 - FireLog - INFO - func:save_fire_layers took: 105.58 ms
2024-03-26 17:21:45,497 - FireLog - INFO - func:merge_rows took: 14.63 ms
2024-03-26 17:21:45,603 - FireLog - INFO - func:save_fire_layers took: 105.25 ms
2024-03-26 17:21:45,704 - FireLog - INFO - func:save_fire_layers took: 99.94 ms
2024-03-26 17:21:45,808 - FireLog - INFO - func:save_fire_layers took: 102.39 ms
2024-03-26 17:21:45,919 - FireLog - INFO - func:save_fire_layers took: 109.17 ms
2024-03-26 17:21:46,020 - FireLog - INFO - func:save_fire_layers took: 99.36 ms
2024-03-26 17:21:46,156 - FireLog - INFO - func:save_fire_layers took: 135.07 ms
2024-03-26 17:21:46,270 - FireLog - INFO - func:save_fire_layers took: 112.90 ms
2024-03-26 17:21:46,373 - FireLog - INFO - f

CPU times: user 1.49 s, sys: 104 ms, total: 1.6 s
Wall time: 4.18 s
